In [1]:
import sys
import pickle
import json
import re
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

In [2]:
def feature_transform(train_file, words, tags):
    
    words_len = len(words)
    tags_len = len(tags)
    row_idx = []
    column_idx = []
    data = []
    y = []
    ctr = 0
    feature_length = words_len + tags_len # Per head

    f = open(train_file)

    for line in f:
            ctr += 1
            if(line.rstrip()):
                line = re.sub("\s+"," ",line)
                line1 = line.split(";")

                a1 = line1[0].split(" ")
                a2 = line1[1].split(" ")
                a3 = line1[2].split(" ")

                if(a1[0] == "H"):
                    column_idx.append(words.index(a1[1]))
                    column_idx.append(tags.index(a1[4]) + words_len)

                elif(a1[0] == "ROOT"):
                    column_idx.append(words.index("ROOT"))
                    column_idx.append(tags.index("ROOT") + words_len)

                row_idx += [ctr-1]*4
                data += [1] *4
                column_idx.append(feature_length + words.index(a2[2]))
                column_idx.append(feature_length + tags.index(a2[5]) + words_len)

                y.append(a3[1])

    f.close()

    X = csr_matrix((data, (row_idx, column_idx)), shape=(ctr,2*(words_len+tags_len)))
    
    
    return X, y

In [3]:
listfile = "data_tokens.json"
f = open(listfile)
data = json.load(f)
f.close()

words = data["words"]
tags = data["tags"]

train_file = 'training_data.txt'
test_file = "testing_data.txt"

X_train, y_train = feature_transform(train_file, words, tags)
X_test, y_test = feature_transform(test_file, words, tags)


model = LinearSVC()
model.fit(X_train, y_train)

pred_train = model.predict(X_train)
pred_test = model.predict(X_test)

In [4]:
print(classification_report(y_train, pred_train))

              precision    recall  f1-score   support

           L       0.77      0.77      0.77     32409
           R       0.71      0.83      0.76     82664
           U       0.87      0.80      0.84    169393

    accuracy                           0.80    284466
   macro avg       0.78      0.80      0.79    284466
weighted avg       0.81      0.80      0.81    284466



In [5]:
print(classification_report(y_test, pred_test))

              precision    recall  f1-score   support

           L       0.75      0.72      0.73      4174
           R       0.66      0.79      0.72     10448
           U       0.85      0.77      0.81     21561

    accuracy                           0.77     36183
   macro avg       0.75      0.76      0.76     36183
weighted avg       0.78      0.77      0.78     36183

